<a href="https://colab.research.google.com/github/chemicoPy/ML-Forex-Market-Prediction/blob/LSTM-(new)/LSTM_on_stock_data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install python-binance
!pip install configparser

In [ ]:
import pandas as pd
import numpy as np
from binance.client import Client
from binance import Client, ThreadedWebsocketManager 
from datetime import datetime, timedelta, date
import matplotlib.pyplot as plt
import configparser
import matplotlib.pyplot as plt
import time
import math
import os
import io
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
from keras.models import Sequential
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from IPython.display import clear_output
from datetime import date, timedelta
import csv
import warnings

In [ ]:

"""Either the person enters the coin name or select from a scrolling bar containing all available coins """
""" Either the person enters the time interval or select from a scrolling bar containing all time intervals """
""" It'd be better if there's a bar to choose from for coins and time intervals separately """

""" available_coins_are = ["BNBUSDT", "ETHUSDT", "ADAUSDT", "DOTUSDT", "BTCUSDT", "FTMUSDT", "DOGEUSDT", "LINKUSDT", "SUSHIUSDT", "SOLUSDT",
                 "IOSTUSDT", "LTCUSDT", "XRPUSDT", "SXPUSDT", "MATICUSDT", "BCHUSDT"] """

coin = str(input("Enter coin name: ").upper())
time_interval = str(input("Enter time interval name e.g 5m for 5-minute: ").lower())

""" For example:
symbol = 'BTCUSDT'
interval = '1m' """



In [ ]:


def consume_data(symbol,interval):

  symbol = str(symbol)
  interval = str(interval)

  TEST_API_KEY = 'cmC20L6OdOe6nfS4iD0T3aIYYbGh53G9nsz1kSO4pd5JsVtVbsycpvzxIwN89AEN'
  TEST_SECRET_KEY = 'JjyurQGV0hcbsEhsdUQoaoaeA85kw33xRrEJywhOJfI6ZNUvT42RMkQK1ed59gyh'

  client = Client(TEST_API_KEY, TEST_SECRET_KEY)
  client.API_URL = 'https://testnet.binance.vision/api'  # To change endpoint URL for test account

  # Preparing Start to End period for klines historical data

  td=timedelta(-10)
  time_now = datetime.now()
  start1 = time_now + td
  start = start1.strftime('%Y-%m-%d- %H:%M:%S')



  td_end = timedelta(hours=1)
  Current_Date = datetime.today() + td_end
  end=Current_Date.strftime ('%Y-%m-%d- %H:%M:%S')


  whole_data = []

  klines = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
  #print(klines)

  df = pd.DataFrame(klines)
  whole_data.append(df)


  #print(start)
  #print(end)

  df = df.rename(columns={0: 'Open time', 1: 'Open', 2: 'High', 3: 'Low', 4: 'Close', 5: 'Volume',
                                          6: 'Close Time', 7: 'Qoute Asset Volume', 8: 'No of Trades', 9: 'Taker buy',
                                          10: 'Taker buy quote asset volume', 11: 'Coin_Name'})

  df.drop("Coin_Name", axis = 1, inplace = True)

  def convertTime(t):
    t = int(t)
    return datetime.fromtimestamp(t)


  df["Open time"]= df["Open time"]/1000
  df["Open time"] = df["Open time"].apply(convertTime)
  df["Close Time"]= df["Close Time"]/1000
  df["Close Time"] = df["Close Time"].apply(convertTime)

  data = df

  return data




In [ ]:
consume_data(coin, time_interval)

In [ ]:
""" Using LSTM model ""

In [ ]:


data = consume_data(coin, time_interval)

#Train-Test split for time-series 

test_ratio = 0.2
training_ratio = 1 - test_ratio

train_size = int(training_ratio * len(data))
test_size = int(test_ratio * len(data))
#print("train_size: " + str(train_size))
#print("test_size: " + str(test_size))

train = data[:train_size][['Close Time', 'Close']]
test = data[train_size:][['Close Time', 'Close']]

  
# Split the time-series data into training seq X and output value Y
def extract_seqX_outcomeY(dataset, N, offset):
    """
    Split time-series into training sequence X and outcome value Y
    Args:
        data - dataset 
        N - window size, e.g., 50 for 50 days of historical stock prices
        offset - position to start the split
    """
    X, y = [], []
    
    for i in range(offset, len(dataset)):
        X.append(dataset[i-N:i])
        y.append(dataset[i])
    
    return np.array(X), np.array(y)

In [ ]:
layer_units = 50
window_size = 50
optimizer = 50 
cur_epochs = 15
cur_batch_size = 20
    
cur_LSTM_pars = {'units': layer_units, 
                     'optimizer': optimizer, 
                     'batch_size': cur_batch_size, 
                     'epochs': cur_epochs
                     }  

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[['Close']])
scaled_data_train = scaled_data[:train.shape[0]]
    
# Using past 50 days’ stock prices for our training to predict the 51th day's closing price.
X_train, y_train = extract_seqX_outcomeY (scaled_data_train, window_size, window_size)

  

In [ ]:
# Building LSTM model

def Run_LSTM(X_train, layer_units=layer_units):     
    inp = Input(shape=(X_train.shape[1], 1))
    
    x = LSTM(units=layer_units, return_sequences=True)(inp)
    x = LSTM(units=layer_units)(x)
    out = Dense(1, activation='linear')(x)
    model = Model(inp, out)

    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  
    return model   

model = Run_LSTM(X_train, layer_units=layer_units)

history = model.fit(X_train, y_train, epochs=cur_epochs, batch_size=cur_batch_size, 
                    verbose=1, validation_split=0.1, shuffle=True)

history.save('lstm_on_stock_data_model.h5')

In [ ]:
# Predicting stock prices using past window_size stock prices

def preprocess_testdat(data=data, scaler=scaler, window_size=window_size, test=test):    
    raw = data['Close'][len(data) - len(test) - window_size:].values
    raw = raw.reshape(-1,1)
    raw = scaler.transform(raw)
    
    X_test = []
    for i in range(window_size, raw.shape[0]):
        X_test.append(raw[i-window_size:i, 0])
        
    X_test = np.array(X_test)
    
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    return X_test

X_test = preprocess_testdat()

predicted_price_ = model.predict(X_test)
predicted_price = scaler.inverse_transform(predicted_price_)

# Plot predicted price vs actual closing price 
test['Predictions_lstm'] = predicted_price

In [ ]:
""" def plot_stock_trend_lstm(train, test):        
    fig = plt.figure(figsize = (20,10))
    plt.plot(train['Close Time'], train['Close'], label = 'Train Closing Price')
    plt.plot(test['Close Time'], test['Close'], label = 'Test Closing Price')
    plt.plot(test['Close Time'], test['Predictions_lstm'], label = 'Predicted Closing Price')
    plt.title('LSTM Model')
    plt.xlabel('Date')
    plt.ylabel('Stock Price ($)')
    plt.legend(loc="upper left")
        
plot_stock_trend_lstm(train, test) """

In [ ]:
predicted_price

In [ ]:
predicted_price[0] # first value in the predicted price array

array([63793.207], dtype=float32)

In [ ]:
predicted_price[-1] # last value in the predicted price array

array([63210.65], dtype=float32)